In [24]:
import numpy as np
import rasterio
from scipy.constants import h, c, k  # Planck constant, speed of light, and Boltzmann constant

In [25]:
# Read raster band data
def read_band(file_path):
    with rasterio.open(file_path) as src:
        band_data = src.read(1).astype(float)
    return band_data, src.meta


In [26]:
# Convert digital numbers to radiance for Landsat TM
def dn_to_radiance_tm(dn, lmax, lmin, qcal_max, qcal_min):
    return ((lmax - lmin) / (qcal_max - qcal_min)) * (dn - qcal_min) + lmin

In [27]:
# Convert digital numbers to radiance for Landsat 8 (OLI/TIRS)
def dn_to_radiance_oli(dn, ml, al):
    return ml * dn + al

In [28]:
# Calculate brightness temperature from radiance
def calculate_brightness_temp(radiance, k1, k2):
    return k2 / np.log((k1 / radiance) + 1)

In [29]:
# Calculate NDVI
def calculate_ndvi(nir, red):
    ndvi = (nir - red) / (nir + red)
    return np.clip(ndvi, -1, 1)

In [30]:
# Calculate proportion of vegetation (Pv)
def calculate_pv(ndvi, ndvi_min=-1, ndvi_max=1):
    return ((ndvi - ndvi_min) / (ndvi_max - ndvi_min)) ** 2

In [31]:
# Calculate land surface emissivity (LSE)
def calculate_emissivity(pv):
    return 0.004 * pv + 0.986

In [32]:
# Calculate land surface temperature (LST)
def calculate_lst(bt, emissivity, wavelength=0.00115):
    p = h * c / k  # Planck's law constant ≈ 14380
    return bt / (1 + (wavelength * bt / p) * np.log(emissivity))

In [33]:
# Load Landsat bands for NDVI calculation (adjust paths accordingly)
red_band, meta = read_band(r"C:\Users\SuperAdmin\Downloads\LC08_L2SP_144045_20201228_20210310_02_T1_SR_B4.TIF")
nir_band, _ = read_band(r"C:\Users\SuperAdmin\Downloads\LC08_L2SP_144045_20201228_20210310_02_T1_SR_B5.TIF")
# Calculate NDVI
ndvi = calculate_ndvi(nir_band, red_band)
# Save NDVI result to a new file
output_path_ndvi = r"D:\pythonCourse05-04-2024\Git_Hub\L8_2020_NDVi.tif"
meta.update(dtype=rasterio.float32)

C:\Users\SuperAdmin\AppData\Local\Temp\ipykernel_13636\2278530811.py:3: RuntimeWarning: invalid value encountered in divide
  ndvi = (nir - red) / (nir + red)


In [34]:
with rasterio.open(output_path_ndvi, 'w', **meta) as dst:
    dst.write(ndvi.astype(rasterio.float32), 1)

# Calculate Pv and emissivity for LST
pv = calculate_pv(ndvi)
emissivity = calculate_emissivity(pv)

# ========== LST Calculation for Landsat 5 (TM) ==========
thermal_band_tm, _ = read_band(r"C:\Users\SuperAdmin\Downloads\LC08_L2SP_144045_20201228_20210310_02_T1_ST_B10.TIF")

In [35]:
# Metadata and constants for Landsat 5 TM (modify based on your metadata file)
L5_LMAX = 15.03
L5_LMIN = 1.238
L5_QCAL_MAX = 255
L5_QCAL_MIN = 1
L5_K1 = 607.76
L5_K2 = 1260.56

In [36]:
# Convert DN to radiance for Landsat 5 TM and then to brightness temperature
radiance_tm = dn_to_radiance_tm(thermal_band_tm, L5_LMAX, L5_LMIN, L5_QCAL_MAX, L5_QCAL_MIN)
bt_tm = calculate_brightness_temp(radiance_tm, L5_K1, L5_K2)


In [37]:
# Calculate LST for Landsat 5 TM
lst_tm = calculate_lst(bt_tm, emissivity)
lst_tm_celsius = lst_tm - 273.15

In [38]:
# Save LST for Landsat 5 TM to a new file
output_path_lst_tm =r"D:\pythonCourse05-04-2024\Git_Hub\L8_2009_LST.tif"
meta.update(dtype=rasterio.float32)

In [39]:
with rasterio.open(output_path_lst_tm, 'w', **meta) as dst:
    dst.write(lst_tm_celsius.astype(rasterio.float32), 1)

# ========== LST Calculation for Landsat 8 (OLI/TIRS) ==========
thermal_band10, _ = read_band(r"C:\Users\SuperAdmin\Downloads\LC08_L2SP_144045_20201228_20210310_02_T1_ST_B10.TIF")

In [40]:
# Metadata and constants for Landsat 8 Band 10
L8_ML_BAND10 = 3.3420E-04
L8_AL_BAND10 = 0.10000
L8_K1_BAND10 = 774.8853
L8_K2_BAND10 = 1321.0789

In [41]:
# Convert DN to radiance for Landsat 8 (OLI/TIRS) using Band 10
radiance_band10 = dn_to_radiance_oli(thermal_band10, L8_ML_BAND10, L8_AL_BAND10)

In [42]:
# Calculate brightness temperature for Landsat 8 Band 10
bt_band10 = calculate_brightness_temp(radiance_band10, L8_K1_BAND10, L8_K2_BAND10)

In [43]:
# Calculate LST for Landsat 8 using brightness temperature
lst_oli = calculate_lst(bt_band10, emissivity)
lst_oli_celsius = lst_oli - 273.15

In [44]:
# Save LST for Landsat 8 to a new file
output_path_lst_oli =r"D:\pythonCourse05-04-2024\Git_Hub\L8_2020_LST.tif"

In [45]:
with rasterio.open(output_path_lst_oli, 'w', **meta) as dst:
    dst.write(lst_oli_celsius.astype(rasterio.float32), 1)
    print("NDVI, LST for Landsat 5, and LST for Landsat 8 calculations are complete.")
print("Outputs saved to:", output_path_ndvi, output_path_lst_tm, output_path_lst_oli)

NDVI, LST for Landsat 5, and LST for Landsat 8 calculations are complete.
Outputs saved to: D:\pythonCourse05-04-2024\Git_Hub\L8_2020_NDVi.tif D:\pythonCourse05-04-2024\Git_Hub\L8_2009_LST.tif D:\pythonCourse05-04-2024\Git_Hub\L8_2020_LST.tif
